In [3]:
! pip install pandas

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import pandas as pd 

data = pd.read_csv("troop_movements_1m.csv")
pd.set_option('display.width', 1000)

print(data.head())

             timestamp  unit_id     unit_type  location_x  location_y  destination_x  destination_y    homeworld
0  2023-06-03 03:19:15   919214  tie_silencer         2.0         5.0              9              3  Aleen Minor
1  2023-02-24 13:50:40  9467154  stormtrooper         9.0         0.0              9              1    Malastare
2  2023-03-29 19:54:55  6585778  tie_silencer         0.0         6.0              5              9      Serenno
3  2023-04-30 00:58:11  3878023  tie_silencer         4.0         2.0              9              9         Tund
4  2023-04-10 22:00:26  5537117         at-st         6.0         8.0              5              8        Skako


In [ ]:
! pip install pyarrow

In [ ]:
! pip install fastparquet

In [7]:
df = pd.DataFrame(data)

df["unit_type"].replace("invalid_unit", "unknown")

0         tie_silencer
1         stormtrooper
2         tie_silencer
3         tie_silencer
4                at-st
              ...     
999995     tie_fighter
999996           at-at
999997     tie_fighter
999998     tie_fighter
999999           at-st
Name: unit_type, Length: 1000000, dtype: object

In [8]:
newDf = df.ffill()

df = df.ffill()

In [10]:
newDf.to_parquet("Troop.parquet.gzip", compression = "gzip")

In [11]:
pd.read_parquet("Troop.parquet.gzip")

,timestamp,unit_id,unit_type,location_x,location_y,destination_x,destination_y,homeworld
0,2023-06-03 03:19:15,919214,tie_silencer,2.0,5.0,9,3,Aleen Minor
1,2023-02-24 13:50:40,9467154,stormtrooper,9.0,0.0,9,1,Malastare
2,2023-03-29 19:54:55,6585778,tie_silencer,0.0,6.0,5,9,Serenno
3,2023-04-30 00:58:11,3878023,tie_silencer,4.0,2.0,9,9,Tund
4,2023-04-10 22:00:26,5537117,at-st,6.0,8.0,5,8,Skako
...,...,...,...,...,...,...,...,...
999995,2023-05-03 18:48:56,3553240,tie_fighter,8.0,4.0,8,1,Chandrila
999996,2023-05-25 13:45:35,1662636,at-at,0.0,3.0,8,6,Trandosha
999997,2023-05-17 12:16:37,2362262,tie_fighter,6.0,8.0,7,9,Kashyyyk
999998,2023-03-17 08:04:26,7669749,tie_fighter,9.0,3.0,3,5,Zolan


In [12]:
import pickle
from flask import Flask, request, jsonify 

app = Flask(__name__)

with open("trained_model.pkl", "rb") as file:
    model = pickle.load(file)

@app.route('/api/predict', methods=['POST'])
def predict():
    # Get the request data
    data = request.get_json(force=True)

     # Ensure the data is a list (even if it's just one dictionary)
    if isinstance(data, dict):
        data = [data]

    # Make a prediction
    prediction = model.predict(newDf)

    # Return the prediction
    return jsonify(prediction.tolist())

if __name__ == '__main__':
    app.run(port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
